In [2]:
from src.utils.load_data import load_data
from src.utils.stratified_train_test_split import stratified_train_test_split
from src.models.AdvancedRegressionModel import AdvancedRegressionModel

from src.evaluation.AbstractEvaluationResults import MetricEnum

In [6]:
metrics = [
    MetricEnum.MSE,
    MetricEnum.MAE,
    MetricEnum.MAPE,
    MetricEnum.R2,
]

In [3]:
X, y = load_data()
X_train, X_test, y_train, y_test = stratified_train_test_split(X, y, 0.3, 42)

In [ ]:
model = AdvancedRegressionModel()
for metric in metrics:
    model.bayesian_optimization(X_train, y_train, X_test, y_test, metric, 100)
    print(f"Optimizing for {metric.name}")
    print(model.score(X_test, y_test))
    print(model.hyperparameters)

In [ ]:
model = AdvancedRegressionModel()
model.bayesian_optimization(X_train, y_train, X_test, y_test, MetricEnum.MAE, 1000)
print(model.score(X_test, y_test))
model.save_hyperparameters("maxed")
model.save_model("maxed")

In [ ]:
print(model.score(X_test, y_test))

In [ ]:
# Regression metrics test
test_model = AdvancedRegressionModel()
test_model.load_hyperparameters("maxed")
test_model.fit(X_train, y_train)
print(test_model.score(X_test, y_test))

# Classification metrics test
from src.utils.IntervalsHandler import IntervalsHandler

interval_function = lambda x: 10 ** (0.25 * x) + 600 * (x + 1) ** 1.15 + 200 * x ** 0.75
interval_handler = IntervalsHandler(y, interval_function)
interval_handler.classify()
interval_handler.reduction()

X_train_class, X_test_class, y_train_class, y_test_class = \
    stratified_train_test_split(X, interval_handler.data, 0.3, 42)

class_test_model = AdvancedRegressionModel()
class_test_model.load_hyperparameters("maxed")
class_test_model.fit(X_train_class, y_train_class)
print(test_model.score_classification(X_test_class, y_test_class, interval_handler))